# 🚀 FinSentiment: Análisis de Sentimiento Financiero en la Nube

Este notebook integra todo el pipeline de FinSentiment para ejecutarse en Google Colab.
Aprovechamos la velocidad de descarga y procesamiento de la nube para evitar limitaciones de hardware local.

### Pasos:
1. Ejecutar instalación de dependencias.
2. Ejecutar las celdas de definición de clases.
3. Ejecutar el análisis final con el ticker de tu elección.

In [ ]:
# @title 1. Instalación de Dependencias
# Instalamos librerías necesarias en el entorno de Colab
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4
!mkdir -p data/raw data/processed reports/figures

In [ ]:
# @title 2. Definición de Módulos (Ingesta, Limpieza, Modelo, Visualización)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# --- MÓDULO 1: INGESTA ---
class SECLoader:
    def __init__(self, data_dir="data", email="usuario@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 1):
        print(f"[>] Descargando reportes para {ticker}...")
        try:
            self.downloader.get("10-K", ticker, limit=amount)
            self.downloader.get("10-Q", ticker, limit=amount)
            print("[+] Descarga completa.")
        except Exception as e:
            print(f"[!] Error en descarga: {e}")

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        # Heurística simple para MD&A (Item 7)
        mda_start_patterns = [
            r'Item\s+7\.\s+Management\'s\s+Discussion',
            r'MANAGEMENT\'S\s+DISCUSSION\s+AND\s+ANALYSIS'
        ]
        mda_end_patterns = [
            r'Item\s+7A\.\s+Quantitative',
            r'Item\s+8\.\s+Financial'
        ]
        
        start_idx = -1
        for p in mda_start_patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match:
                start_idx = match.start()
                break
        
        if start_idx == -1: return ""
        
        end_idx = -1
        for p in mda_end_patterns:
            match = re.search(p, text[start_idx:], re.IGNORECASE)
            if match:
                end_idx = start_idx + match.start()
                break
        
        if end_idx != -1:
            return text[start_idx:end_idx]
        return text[start_idx:start_idx+100000] # Fallback

    def process_local_filings(self, ticker: str):
        raw_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        print(f"[*] Procesando archivos locales para {ticker}...")
        
        processed_count = 0
        for root, dirs, files in os.walk(raw_path):
            for file in files:
                if file.endswith(".html") or file.endswith(".txt"):
                    full_path = os.path.join(root, file)
                    try:
                        with open(full_path, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                        
                        mda = self.extract_mda(content)
                        if len(mda) > 1000:
                            doc_type = "10-K" if "10-K" in root else "10-Q"
                            accession = os.path.basename(os.path.dirname(full_path))
                            save_path = os.path.join(self.data_dir, "processed", f"{ticker}_{doc_type}_{accession}_MDA.txt")
                            with open(save_path, 'w', encoding='utf-8') as f_out:
                                f_out.write(mda)
                            processed_count += 1
                    except Exception as e:
                        pass
        print(f"[+] Procesados {processed_count} reportes.")

# --- MÓDULO 2: PREPROCESAMIENTO ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        text = "".join(ch for ch in text if ch.isprintable())
        return text.strip()
    
    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        return [s.strip() for s in sentences if len(s.split()) >= 4]

# --- MÓDULO 3: MODELO FINBERT ---
class FinBertModel:
    def __init__(self):
        print("[*] Cargando modelo FinBERT (esto es rápido en Colab)...\n")
        self.model_name = "ProsusAI/finbert"
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.model = BertForSequenceClassification.from_pretrained(self.model_name)
        # Mover a GPU si está disponible
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        print(f"[+] Modelo cargado en: {self.device}")
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict_batch(self, sentences):
        if not sentences: return pd.DataFrame()
        inputs = self.tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        probs_np = probs.cpu().numpy()
        results = []
        for i, s in enumerate(sentences):
            max_idx = np.argmax(probs_np[i])
            results.append({
                "sentence": s,
                "positive": probs_np[i][0],
                "negative": probs_np[i][1],
                "neutral": probs_np[i][2],
                "sentiment_label": self.labels[max_idx]
            })
        return pd.DataFrame(results)

# --- MÓDULO 4: VISUALIZACIÓN ---
def plot_trend(df, ticker):
    if df.empty: return
    df = df.sort_values('date')
    df['ma'] = df['sentiment_score'].rolling(window=3, min_periods=1).mean()
    
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['sentiment_score'], 'o-', label='Sentiment Score', color='#2E86AB')
    plt.plot(df['date'], df['ma'], '--', label='Media Móvil', color='red')
    plt.axhline(0, color='gray', alpha=0.5)
    plt.title(f'Evolución del Sentimiento: {ticker}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# @title 3. EJECUTAR ANÁLISIS
TICKER = "ADBE" # @param {type:"string"}
NUM_REPORTS = 2 # @param {type:"integer"}

# 1. Ingesta
loader = SECLoader()
loader.download_filings(TICKER, amount=NUM_REPORTS)
loader.process_local_filings(TICKER)

# 2. Preprocesamiento y Análisis
preprocessor = TextPreprocessor()
model = FinBertModel()

processed_dir = "data/processed"
files = [f for f in os.listdir(processed_dir) if TICKER in f]

results_agg = []

for file in files:
    print(f"\nAnalizando: {file}")
    with open(os.path.join(processed_dir, file), 'r') as f:
        text = f.read()
    
    sentences = preprocessor.split_sentences(preprocessor.clean_text(text))
    print(f" - Oraciones: {len(sentences)}")
    
    # Batch processing simple (de 10 en 10 para no saturar RAM de Colab si es muy grande)
    all_preds = []
    chunk_size = 16
    for i in range(0, len(sentences), chunk_size):
        batch = sentences[i:i+chunk_size]
        preds = model.predict_batch(batch)
        all_preds.append(preds)
    
    if all_preds:
        df_file = pd.concat(all_preds)
        net_score = df_file['positive'].mean() - df_file['negative'].mean()
        results_agg.append({
            'date': datetime.now(), # Proxy, en real usaríamos fecha del fila
            'sentiment_score': net_score
        })
        print(f" - Score Neto: {net_score:.3f}")

# 3. Visualización
if results_agg:
    df_final = pd.DataFrame(results_agg)
    # Simular fechas para demo si todas son 'hoy'
    df_final['date'] = pd.date_range(end=datetime.now(), periods=len(df_final), freq='3M')
    
    plot_trend(df_final, TICKER)